# LDA

In [21]:
import pickle
import pandas as pd

with open('./results/HAS_e.pickle', 'rb') as f:
    HAS_e = pickle.load(f)
HAS_t = pd.read_parquet('./results/HAS_t.parquet')

In [22]:
from time import time

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(embedding):
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=embedding['corpus'],
            id2word=embedding['id2word'],
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=embedding['T'], dictionary=embedding['id2word'], coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, embedding['id2word']))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

In [23]:
def save_models_lda():
    r_time = []
    for v in HAS_e:
        model = lda(HAS_e[v])
        model['model'].save(f'./results/models_lda/{v}')
        r_time.append(model['time'])
    HAS_t['lda_training'] = pd.Series(r_time)

In [24]:
save_models_lda()

[I 2023-08-19 11:45:40,114] A new study created in memory with name: no-name-d7d9a361-d19d-4a0f-afca-c1b13e19a34c
[I 2023-08-19 11:45:48,008] Trial 0 finished with value: 0.08670385901956995 and parameters: {'num_topics': 99, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'symmetric'}. Best is trial 0 with value: 0.08670385901956995.
[I 2023-08-19 11:45:56,146] Trial 1 finished with value: 0.18891257402127407 and parameters: {'num_topics': 77, 'alpha_categorical': 'symmetric', 'eta_categorical': 'scalar', 'eta': 0.05798333657720763}. Best is trial 1 with value: 0.18891257402127407.


In [25]:
HAS_t

,variant,segmentation,preprocess,lda_embedding,lda_training,bertopic_training
0,HWN,0.001003,2.34099,0.003,19.633259,6.656391


# BERTopic

In [26]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    path_prefix = './results/models_bertopic/'
    r_time = []
    for v in HAS_e:
        t_start = time()
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(HAS_e[v]['text'])
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    HAS_t['bertopic_training'] = pd.Series(r_time)

In [28]:
save_models_bertopic()

In [29]:
HAS_t

,variant,segmentation,preprocess,lda_embedding,lda_training,bertopic_training
0,HWN,0.001003,2.34099,0.003,19.633259,7.961455


In [30]:
HAS_t.to_parquet('./results/HAS_t.parquet')